# Arquitecturas de redes convolucionales

Es sumamente dificil y tedioso desarrollar arquitecturas para cada problema que se nos presente, como vimos anteriormente, algunas de estas arquitecturas pueden ser sumamente grandes y complejas

<https://keras.io/api/applications/>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install pycm
# !pip install livelossplot

In [ ]:
%matplotlib inline
import sys
import pathlib
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from livelossplot import PlotLossesKerasTF

os.environ["CUDA_VISIBLE_DEVICES"]="1"

SEED = 199510
np.random.seed(SEED)
tf.random.set_seed(SEED)

ROOT_DIR = pathlib.Path().resolve().parent

# Descomentar para correr en Google Colab usando Drive
# ROOT_DIR = pathlib.Path("drive/MyDrive/curso_cisc_2022")
# sys.path.append("/content/drive/MyDrive/curso_cisc_2022/Notebooks")

DATA_DIR = ROOT_DIR / "Data" / "smear2005"
print(ROOT_DIR)

import utils

In [ ]:
dataset = pd.read_csv(f"{DATA_DIR}/dataset.csv")
CLASS_COLUMN = "Class_cat_7"

In [ ]:
class_names = dataset[CLASS_COLUMN].unique()
num_classes = len(class_names)
print(f"Número de clases: {num_classes}")
print(f"Clases: {class_names}")

In [ ]:
IMG_HEIGHT = 100
IMG_WIDTH = 100
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
BATCH_SIZE = 256
EPOCHS = 100
LEARNING_RATE = 0.001
OPT = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
METRICS = [
    "accuracy", 
    tf.keras.metrics.Precision(name="precision"), 
    tf.keras.metrics.Recall(name="recall")
    ]
LOSS = "binary_crossentropy" if num_classes == 2 else "categorical_crossentropy"
OUTPUT_NEURONS = 1 if num_classes == 2 else num_classes
OUTPUT_ACTIVATION = "sigmoid" if num_classes == 2 else "softmax"
SPLIT = 0.2

In [ ]:
train, val = train_test_split(dataset, test_size=SPLIT, random_state=SEED, stratify=dataset[CLASS_COLUMN])

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

val_datagen =tf.keras.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_dataframe(
                                        train,
                                        None,
                                        x_col='filename',
                                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                                        y_col=CLASS_COLUMN,
                                        batch_size=BATCH_SIZE,
                                        seed=SEED,
                                        class_mode='binary' if num_classes == 2 else 'categorical'
                                        )

val_generator = val_datagen.flow_from_dataframe(
                                        val,
                                        None,
                                        x_col='filename',
                                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                                        y_col=CLASS_COLUMN,
                                        batch_size=1,
                                        seed=SEED,
                                        class_mode='binary' if num_classes == 2 else 'categorical'
                                        )

In [ ]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip(input_shape=IMG_SHAPE),
    tf.keras.layers.RandomRotation(0.3, fill_mode="constant"),
    tf.keras.layers.RandomZoom(0.2, fill_mode="constant"),
  ]
)

In [ ]:
plt.figure(figsize=(10,10))
batch = train_generator.next()
image_ = batch[0].astype('uint8')
for i in range(9):
  plt.subplot(3, 3, i + 1)
  augmented = data_augmentation(image_[0])
  plt.imshow(data_augmentation(image_[0]).numpy().astype('uint8'))
  plt.axis("off")
plt.show()

In [ ]:
base_model = tf.keras.applications.VGG16(weights=None, include_top=False, input_shape=IMG_SHAPE)
preprocess_input = tf.keras.applications.vgg16.preprocess_input
base_model.trainable = True

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Lambda(preprocess_input, input_shape=IMG_SHAPE),
  data_augmentation,
  base_model,
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(OUTPUT_NEURONS, activation=OUTPUT_ACTIVATION)
])

model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, expand_nested=True, show_layer_names=True)

In [ ]:
print(model.summary())

In [ ]:
history = model.fit(
  train_generator,
  validation_data=val_generator,
  callbacks=[PlotLossesKerasTF()],
  epochs=EPOCHS,
  steps_per_epoch=train_generator.n // train_generator.batch_size,
  validation_steps=val_generator.n // val_generator.batch_size
)

In [ ]:
evaluations = model.evaluate(val_generator, verbose=0)

for evaluation, metric_name in zip(evaluations, model.metrics_names):
  print(f"{metric_name}: {evaluation}")

In [ ]:
base_model = tf.keras.applications.VGG19(weights=None, include_top=False, input_shape=IMG_SHAPE)
preprocess_input = tf.keras.applications.vgg19.preprocess_input
base_model.trainable = True

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Lambda(preprocess_input, input_shape=IMG_SHAPE),
  data_augmentation,
  base_model,
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(OUTPUT_NEURONS, activation=OUTPUT_ACTIVATION)
])

model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, expand_nested=True, show_layer_names=True)

In [ ]:
print(model.summary())

In [ ]:
history = model.fit(
  train_generator,
  validation_data=val_generator,
  callbacks=[PlotLossesKerasTF()],
  epochs=EPOCHS,
  steps_per_epoch=train_generator.n // train_generator.batch_size,
  validation_steps=val_generator.n // val_generator.batch_size
)

In [ ]:
evaluations = model.evaluate(val_generator, verbose=0)

for evaluation, metric_name in zip(evaluations, model.metrics_names):
  print(f"{metric_name}: {evaluation}")

In [ ]:
base_model = tf.keras.applications.resnet.ResNet50(weights=None, include_top=False, input_shape=IMG_SHAPE)
preprocess_input = tf.keras.applications.resnet.preprocess_input
base_model.trainable = True

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Lambda(preprocess_input, input_shape=IMG_SHAPE),
  data_augmentation,
  base_model,
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(OUTPUT_NEURONS, activation=OUTPUT_ACTIVATION)
])

model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, expand_nested=True, show_layer_names=True)

In [ ]:
print(model.summary())

In [ ]:
history = model.fit(
  train_generator,
  validation_data=val_generator,
  callbacks=[PlotLossesKerasTF()],
  epochs=EPOCHS,
  steps_per_epoch=train_generator.n // train_generator.batch_size,
  validation_steps=val_generator.n // val_generator.batch_size
)

In [ ]:
evaluations = model.evaluate(val_generator, verbose=0)

for evaluation, metric_name in zip(evaluations, model.metrics_names):
  print(f"{metric_name}: {evaluation}")